In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression

from hyperopt import hp
from hyperopt import Trials

from lightgbm import *

try:
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
except:
    !pip install --upgrade shap-hypetune
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA

import warnings
warnings.simplefilter('ignore')

In [2]:
X_clf, y_clf = make_classification(n_samples=6000, n_features=20, n_classes=2, 
                                   n_informative=4, n_redundant=6, random_state=0)

X_clf_train, X_clf_valid, y_clf_train, y_clf_valid = train_test_split(
    X_clf, y_clf, test_size=0.3, shuffle=False)

X_regr, y_regr = make_classification(n_samples=6000, n_features=20,
                                     n_informative=7, random_state=0)

X_regr_train, X_regr_valid, y_regr_train, y_regr_valid = train_test_split(
    X_regr, y_regr, test_size=0.3, shuffle=False)

In [3]:
param_grid = {
    'learning_rate': [0.2, 0.1],
    'num_leaves': [25, 35],
    'max_depth': [10, 12]
}

param_dist = {
    'learning_rate': stats.uniform(0.09, 0.25),
    'num_leaves': stats.randint(20,40),
    'max_depth': [10, 12]
}

param_dist_hyperopt = {
    'max_depth': 15 + hp.randint('num_leaves', 5), 
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}


regr_lgbm = LGBMRegressor(n_estimators=150, random_state=0, n_jobs=-1)
clf_lgbm = LGBMClassifier(n_estimators=150, random_state=0, n_jobs=-1)

# Hyperparameters Tuning

In [4]:
### HYPERPARAM TUNING WITH GRID-SEARCH ###

model = BoostSearch(clf_lgbm, param_grid=param_grid)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

In [5]:
model.estimator_, model.best_params_, model.best_score_

In [6]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.predict_proba(X_clf_valid).shape)

In [7]:
### HYPERPARAM TUNING WITH RANDOM-SEARCH ###

model = BoostSearch(
    regr_lgbm, param_grid=param_dist,
    n_iter=8, sampling_seed=0
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [8]:
model.estimator_, model.best_params_, model.best_score_

In [9]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.predict(X_regr_valid, pred_contrib=True).shape)

In [10]:
### HYPERPARAM TUNING WITH HYPEROPT ###

model = BoostSearch(
    regr_lgbm, param_grid=param_dist_hyperopt,
    n_iter=8, sampling_seed=0
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)

In [11]:
model.estimator_, model.best_params_, model.best_score_

In [12]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.predict(X_regr_valid, pred_contrib=True).shape)

# Features Selection

In [13]:
### BORUTA ###

model = BoostBoruta(clf_lgbm, max_iter=200, perc=100)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

In [14]:
model.estimator_, model.n_features_

In [15]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

In [16]:
### RECURSIVE FEATURE ELIMINATION (RFE) ###

model = BoostRFE(regr_lgbm, min_features_to_select=1, step=1)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [17]:
model.estimator_, model.n_features_

In [18]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

In [19]:
### RECURSIVE FEATURE ADDITION (RFA) ###

model = BoostRFA(regr_lgbm, min_features_to_select=1, step=1)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [20]:
model.estimator_, model.n_features_

In [21]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

# Features Selection with SHAP

In [22]:
### BORUTA SHAP ###

model = BoostBoruta(
    clf_lgbm, max_iter=200, perc=100,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

In [23]:
model.estimator_, model.n_features_

In [24]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

In [25]:
### RECURSIVE FEATURE ELIMINATION (RFE) SHAP ###

model = BoostRFE(
    regr_lgbm, min_features_to_select=1, step=1,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [26]:
model.estimator_, model.n_features_

In [27]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

In [28]:
### RECURSIVE FEATURE ADDITION (RFA) SHAP ###

model = BoostRFA(
    regr_lgbm, min_features_to_select=1, step=1,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [29]:
model.estimator_, model.n_features_

In [30]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

# Hyperparameters Tuning + Features Selection

In [31]:
### HYPERPARAM TUNING WITH GRID-SEARCH + BORUTA ###

model = BoostBoruta(clf_lgbm, param_grid=param_grid, max_iter=200, perc=100)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

In [32]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

In [33]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

In [34]:
### HYPERPARAM TUNING WITH RANDOM-SEARCH + RECURSIVE FEATURE ELIMINATION (RFE) ###

model = BoostRFE(
    regr_lgbm, param_grid=param_dist, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [35]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

In [36]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

In [37]:
### HYPERPARAM TUNING WITH HYPEROPT + RECURSIVE FEATURE ADDITION (RFA) ###

model = BoostRFA(
    regr_lgbm, param_grid=param_dist_hyperopt, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)

In [38]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

In [39]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

# Hyperparameters Tuning + Features Selection with SHAP

In [40]:
### HYPERPARAM TUNING WITH GRID-SEARCH + BORUTA SHAP ###

model = BoostBoruta(
    clf_lgbm, param_grid=param_grid, max_iter=200, perc=100,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)

In [41]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

In [42]:
(model.score(X_clf_valid, y_clf_valid), 
 model.predict(X_clf_valid).shape, 
 model.transform(X_clf_valid).shape,
 model.predict_proba(X_clf_valid).shape)

In [43]:
### HYPERPARAM TUNING WITH RANDOM-SEARCH + RECURSIVE FEATURE ELIMINATION (RFE) SHAP ###

model = BoostRFE(
    regr_lgbm, param_grid=param_dist, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_regr_train, y_regr_train, eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0)

In [44]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

In [45]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

In [46]:
### HYPERPARAM TUNING WITH HYPEROPT + RECURSIVE FEATURE ADDITION (RFA) SHAP ###

model = BoostRFA(
    regr_lgbm, param_grid=param_dist_hyperopt, min_features_to_select=1, step=1,
    n_iter=8, sampling_seed=0,
    importance_type='shap_importances', train_importance=False
)
model.fit(
    X_regr_train, y_regr_train, trials=Trials(), 
    eval_set=[(X_regr_valid, y_regr_valid)], early_stopping_rounds=6, verbose=0
)

In [47]:
model.estimator_, model.best_params_, model.best_score_, model.n_features_

In [48]:
(model.score(X_regr_valid, y_regr_valid), 
 model.predict(X_regr_valid).shape, 
 model.transform(X_regr_valid).shape,
 model.predict(X_regr_valid, pred_contrib=True).shape)

# CUSTOM EVAL METRIC SUPPORT

In [49]:
from sklearn.metrics import roc_auc_score

def AUC(y_true, y_hat):
    return 'auc', roc_auc_score(y_true, y_hat), True

In [50]:
model = BoostRFE(
    LGBMClassifier(n_estimators=150, random_state=0, metric="custom"), 
    param_grid=param_grid, min_features_to_select=1, step=1,
    greater_is_better=True
)
model.fit(
    X_clf_train, y_clf_train, 
    eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0, 
    eval_metric=AUC
)

# CATEGORICAL FEATURE SUPPORT

In [51]:
categorical_feature = [0,1,2]

X_clf_train[:,categorical_feature] = (X_clf_train[:,categorical_feature]+100).clip(0).astype(int)
X_clf_valid[:,categorical_feature] = (X_clf_valid[:,categorical_feature]+100).clip(0).astype(int)

In [52]:
### MANUAL PASS categorical_feature WITH NUMPY ARRAYS ###

model = BoostRFE(clf_lgbm, param_grid=param_grid, min_features_to_select=1, step=1)
model.fit(
    X_clf_train, y_clf_train, 
    eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0,
    categorical_feature=categorical_feature
)

In [53]:
X_clf_train = pd.DataFrame(X_clf_train)
X_clf_train[categorical_feature] = X_clf_train[categorical_feature].astype('category')

X_clf_valid = pd.DataFrame(X_clf_valid)
X_clf_valid[categorical_feature] = X_clf_valid[categorical_feature].astype('category')

In [54]:
### PASS category COLUMNS IN PANDAS DF ###

model = BoostRFE(clf_lgbm, param_grid=param_grid, min_features_to_select=1, step=1)
model.fit(X_clf_train, y_clf_train, eval_set=[(X_clf_valid, y_clf_valid)], early_stopping_rounds=6, verbose=0)